In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os
import gc
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
# import keras.backend as K
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Conv1D, MaxPooling1D, GRU, Lambda
from tensorflow.keras.models import Model, Sequential, load_model

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 10
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))

es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


In [3]:
from tensorflow.keras.layers import add

class LSTNet(keras.Model):
    def __init__(self):
        super(LSTNet, self).__init__()
        self.P = 120 # data length
        self.m = 32 # data width
        self.hidR = 32
        self.hidC = 64
        self.hidS = 1
        self.ks = 3
        self.skip = 2
        self.pt = int((self.P-self.ks)/self.skip)
        self.hw = 3
        self.output_dim = 1
        
        self.model = self.make_model()
        
#         self.loss_fn = tf.keras.losses.MeanSquaredError()
#         self.optim = optimizers.Adam()
        
    def compile(self, optimizer, loss):
        super(LSTNet, self).compile()
        self.loss_functions = loss
        self.optimizer = optimizer
        self._is_compiled = True

    def make_model(self):
        activation = mish
        
        x = Input(shape=(self.P, self.m))

        # CNN
        c = Conv1D(self.hidC, self.ks, activation='relu')(x)

        # RNN
        r = GRU(self.hidR)(c)
        r = Lambda(lambda k: tf.reshape(k, (-1, self.hidR)))(r)

        # skip-RNN
        if self.skip > 0:
            # c: batch_size*steps*filters, steps=P-ks
            s = Lambda(lambda k: k[:, int(-self.pt*self.skip):, :])(c)
            s = Lambda(lambda k: tf.reshape(k, (-1, self.pt, self.skip, self.hidC)))(s)
            s = Lambda(lambda k: tf.transpose(k, (0,2,1,3)))(s)
            s = Lambda(lambda k: tf.reshape(k, (-1, self.pt, self.hidC)))(s)

            s = GRU(self.hidS)(s)
            s = Lambda(lambda k: tf.reshape(k, (-1, self.skip*self.hidS)))(s)
            r = Concatenate()([r,s])
        
        res = Dense(self.m)(r)

        # highway
        if self.hw > 0:
            z = Lambda(lambda k: k[:, -self.hw:, :])(x)
            z = Lambda(lambda k: tf.transpose(k, (0,2,1)))(z)
            z = Lambda(lambda k: tf.reshape(k, (-1, self.hw)))(z)
            z = Dense(1)(z)
            z = Lambda(lambda k: tf.reshape(k, (-1, self.m)))(z)
            res = add([res, z])

        res = Dense(self.output_dim)(res)
        model = Model(inputs=x, outputs=res)
        return model
    
#     def call(self, x):
#         return self.model(x)
    
    def train_step(self, data):
        x, y = data
        batch_size = tf.shape(x)[0]
        
        with tf.GradientTape() as tape:
            pred = self.model(x, training=True)
            loss = self.loss_functions(y, pred)
            
        grads = tape.gradient(loss, self.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
        
        return {'loss': loss}

In [4]:
net = LSTNet()

In [5]:
net.model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 32)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 118, 64)      6208        input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 116, 64)      0           conv1d[0][0]                     
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 58, 2, 64)    0           lambda_1[0][0]                   
_______________________________________________________________________________________

In [6]:
X = np.random.normal(0, 1, (1000, 120, 32))
y = np.random.normal(0, 1, (1000, 1))

In [7]:
net.compile(optimizer=optimizers.Adam(), loss=keras.losses.MeanSquaredError())

In [8]:
net.fit(X, y, 
       epochs = 10)

Epoch 1/10
32/32 [==============================] - 1s 16ms/step - loss: 1.1287
Epoch 2/10
32/32 [==============================] - 0s 14ms/step - loss: 0.9030
Epoch 3/10
32/32 [==============================] - 0s 14ms/step - loss: 0.8530
Epoch 4/10
32/32 [==============================] - 0s 16ms/step - loss: 0.7710
Epoch 5/10
32/32 [==============================] - 0s 15ms/step - loss: 0.6636
Epoch 6/10
32/32 [==============================] - 0s 15ms/step - loss: 0.5545
Epoch 7/10
32/32 [==============================] - 0s 15ms/step - loss: 0.4240
Epoch 8/10
32/32 [==============================] - 0s 15ms/step - loss: 0.2760
Epoch 9/10
32/32 [==============================] - 0s 15ms/step - loss: 0.1795
Epoch 10/10
32/32 [==============================] - 1s 16ms/step - loss: 0.1079
